In [1]:
import json
from pathlib import Path
from functools import reduce

import pandas as pd

from IPython.display import display

In [2]:
ARTICLES_DIR = "../../data/test/user_study/articles/"
STOCKS_PATH = "../../data/processed/stocks/revolut.2021-07-05.complete.stocks.jsonl"

###  Sorting out paths

In [3]:
def key(path):
    stem = path.stem
    if stem.startswith("oos"):
        return int(stem[-1])
    else:
        return int(stem)

In [4]:
# Sorting out paths
article_paths = sorted(list(Path(ARTICLES_DIR).glob("*.json")), key=key)
article_paths

[PosixPath('../../data/test/user_study/articles/0.json'),
 PosixPath('../../data/test/user_study/articles/oos_0.json'),
 PosixPath('../../data/test/user_study/articles/1.json'),
 PosixPath('../../data/test/user_study/articles/oos_1.json'),
 PosixPath('../../data/test/user_study/articles/oos_2.json'),
 PosixPath('../../data/test/user_study/articles/3.json'),
 PosixPath('../../data/test/user_study/articles/oos_3.json'),
 PosixPath('../../data/test/user_study/articles/4.json'),
 PosixPath('../../data/test/user_study/articles/oos_4.json'),
 PosixPath('../../data/test/user_study/articles/oos_5.json'),
 PosixPath('../../data/test/user_study/articles/5.json'),
 PosixPath('../../data/test/user_study/articles/oos_6.json'),
 PosixPath('../../data/test/user_study/articles/6.json'),
 PosixPath('../../data/test/user_study/articles/oos_7.json'),
 PosixPath('../../data/test/user_study/articles/7.json'),
 PosixPath('../../data/test/user_study/articles/8.json'),
 PosixPath('../../data/test/user_study/a

### Inverting articles dict for relevant articles

In [5]:
def read_json(path):
    with open(path, "r") as fp:
        return json.load(fp)


def read_jsonl(path):
    with open(path, "r") as fp:
        for line in fp:
            yield json.loads(line)

In [6]:
stocks

NameError: name 'stocks' is not defined

In [8]:
articles = dict()
stocks = dict()

for path in article_paths:
    article = read_json(path)
    articles[article["doc_id"]] = article
    for stock in article["stock_matches"]:
        ticker = stock["ticker_symbol"]
        if ticker not in stocks.keys():
            stocks[ticker] = stock
            stocks[ticker]["in_articles"] = list()
        stocks[ticker]["in_articles"].append({k:v for k,v in article.items()
                                              if k in ["doc_id", "title", "url", "summary"]})

len(stocks.keys())

246

In [10]:
stocks["LUMN"]

{'index': 439,
 'stock_name': 'Lumen Technologies',
 'ticker_symbol': 'LUMN',
 'sector': 'Communications',
 'industry': 'Specialty Telecommunications',
 'comment': 'Lumen Technologies (formerly CenturyLink) is an American telecommunications company headquartered in Monroe, Louisiana, that offers communications, network services, security, cloud solutions, voice, and managed services. The company is a member of the S&P 500 index and the Fortune 500. Its communications services include local and long-distance voice, broadband, Multi-Protocol Label Switching (MPLS), private line (including special access), Ethernet, hosting (including cloud hosting and managed hosting), data integration, video, network, public access, Voice over Internet Protocol (VoIP), information technology, and other ancillary services. Lumen also serves global enterprise customers across North America, Latin America, EMEA (Europe, Middle East, and Africa), and Asia Pacific.',
 'score': 0.7538793208249759,
 'in_articl

In [12]:
stocks_df = pd.DataFrame.from_dict(stocks, orient="index")
stock_matches_counts = {k:len(v["in_articles"]) for k, v in stocks.items()}

In [13]:
stocks_df["in_articles_count"] = stocks_df.in_articles.apply(len)

In [14]:
with pd.option_context("display.max_rows", None):
    display(stocks_df.groupby(["sector", "industry"]).in_articles_count.sum())

sector                  industry                        
Commercial Services     Advertising/Marketing Services       1
                        Financial Publishing/Services        1
Communications          Major Telecommunications             1
                        Specialty Telecommunications         2
                        Telecommunications Equipment         2
                        Wireless Telecommunications          1
Consumer Durables       Electronics/Appliances               2
                        Homebuilding                         1
                        Motor Vehicles                      15
                        Recreational Products                1
Consumer Non-Durables   Apparel/Footwear                     4
                        Beverages: Alcoholic                 4
                        Beverages: Non-Alcoholic             1
                        Food: Major Diversified              2
                        Household/Personal Care              

In [15]:
test_articles_top_tickers = reduce(set.union, [set(v["ticker_top"]) for v in articles.values() 
                                               if "ticker_top" in v.keys()])
test_articles_top_tickers

{'AAL',
 'AAPL',
 'ABEV',
 'AMC',
 'AMD',
 'BABA',
 'BBD',
 'C',
 'CCL',
 'CVX',
 'DIDI',
 'EDU',
 'ET',
 'F',
 'FCX',
 'INTC',
 'IQ',
 'ITUB',
 'KSS',
 'M',
 'MRNA',
 'NCLH',
 'NIO',
 'NVDA',
 'PBR',
 'PFE',
 'PLUG',
 'SOFI',
 'T',
 'TME',
 'TSLA',
 'TSM',
 'UBER',
 'VALE',
 'VIPS',
 'X',
 'XOM'}

In [16]:
stocks_matches = dict()
keys = ["stock_name", "ticker_symbol", "sector", "industry", "comment"]
for i, stock_ in enumerate(read_jsonl(STOCKS_PATH)):
    ticker_symbol_ = stock_["ticker_symbol"]
    if ticker_symbol_ in test_articles_top_tickers:
        stocks_matches[ticker_symbol_] = dict()
        stocks_matches[ticker_symbol_]["index"] = i
        stocks_matches[ticker_symbol_].update({k:v for k,v in stock_.items()
                                               if k in keys})
stocks_matches

{'AMD': {'index': 11,
  'stock_name': 'Advanced Micro Devices',
  'ticker_symbol': 'AMD',
  'sector': 'Electronic Technology',
  'industry': 'Semiconductors',
  'comment': "Advanced Micro Devices, Inc. (AMD) is an American multinational semiconductor company based in Santa Clara, California, that develops computer processors and related technologies for business and consumer markets. While it initially manufactured its own processors, the company later outsourced its manufacturing, a practice known as going fabless, after GlobalFoundries was spun off in 2009. AMD's main products include microprocessors, motherboard chipsets, embedded processors and graphics processors for servers, workstations, personal computers and embedded system applications."},
 'BABA': {'index': 23,
  'stock_name': 'Alibaba',
  'ticker_symbol': 'BABA',
  'sector': 'Retail Trade',
  'industry': 'Internet Retail',
  'comment': 'Alibaba Group Holding Limited, also known as Alibaba Group and Alibaba.com, is a Chinese

In [14]:
top_sectors_industries = [(stock["sector"], stock["industry"]) for stock in stocks_matches.values()
                          if stock["ticker_symbol"] in test_articles_top_tickers]

print(len(top_sectors_industries))
print(len(set(top_sectors_industries)))

top_sectors_industries = set(top_sectors_industries)
top_sectors_industries

37
23


{('Communications', 'Major Telecommunications'),
 ('Consumer Durables', 'Motor Vehicles'),
 ('Consumer Non-Durables', 'Beverages: Alcoholic'),
 ('Consumer Services', 'Hotels/Resorts/Cruise lines'),
 ('Consumer Services', 'Movies/Entertainment'),
 ('Consumer Services', 'Other Consumer Services'),
 ('Electronic Technology', 'Electronic Components'),
 ('Electronic Technology', 'Semiconductors'),
 ('Electronic Technology', 'Telecommunications Equipment'),
 ('Energy Minerals', 'Integrated Oil'),
 ('Finance', 'Finance/Rental/Leasing'),
 ('Finance', 'Financial Conglomerates'),
 ('Finance', 'Major Banks'),
 ('Health Technology', 'Biotechnology'),
 ('Health Technology', 'Pharmaceuticals: Major'),
 ('Industrial Services', 'Oil & Gas Pipelines'),
 ('Non-Energy Minerals', 'Other Metals/Minerals'),
 ('Non-Energy Minerals', 'Steel'),
 ('Retail Trade', 'Department Stores'),
 ('Retail Trade', 'Internet Retail'),
 ('Technology Services', 'Internet Software/Services'),
 ('Technology Services', 'Packaged

In [15]:
sectors_industries = stocks_df.groupby(["sector", "industry"])
sect_ind_not_present = set(sectors_industries.indices.keys()) - top_sectors_industries

print(len(sect_ind_not_present))
sect_ind_not_present

51


{('Commercial Services', 'Advertising/Marketing Services'),
 ('Commercial Services', 'Financial Publishing/Services'),
 ('Communications', 'Specialty Telecommunications'),
 ('Communications', 'Telecommunications Equipment'),
 ('Communications', 'Wireless Telecommunications'),
 ('Consumer Durables', 'Electronics/Appliances'),
 ('Consumer Durables', 'Homebuilding'),
 ('Consumer Durables', 'Recreational Products'),
 ('Consumer Non-Durables', 'Apparel/Footwear'),
 ('Consumer Non-Durables', 'Beverages: Non-Alcoholic'),
 ('Consumer Non-Durables', 'Food: Major Diversified'),
 ('Consumer Non-Durables', 'Household/Personal Care'),
 ('Consumer Services', 'Broadcasting'),
 ('Consumer Services', 'Cable/Satellite TV'),
 ('Consumer Services', 'Restaurants'),
 ('Electronic Technology', 'Aerospace & Defense'),
 ('Electronic Technology', 'Computer Communications'),
 ('Electronic Technology', 'Computer Peripherals'),
 ('Electronic Technology', 'Computer Processing Hardware'),
 ('Electronic Technology', 

In [16]:
for key, group in sectors_industries:
    if key in sect_ind_not_present:
        group = group.sort_values("in_articles_count", ascending=False)
        print(key)
        display(group)
        top_match = group.head(1).to_dict(orient="index")
        top_match.pop("in_articles_count", None)
        
        stocks_matches.update(top_match)

len(stocks_matches)

('Commercial Services', 'Advertising/Marketing Services')


,index,stock_name,ticker_symbol,sector,industry,comment,score,in_articles,in_articles_count
GRPN,326,Groupon,GRPN,Commercial Services,Advertising/Marketing Services,Groupon is an American global e-commerce marke...,0.726922,"[{'doc_id': 0, 'title': 'AT&T CFO Pascal Desro...",1


('Commercial Services', 'Financial Publishing/Services')


,index,stock_name,ticker_symbol,sector,industry,comment,score,in_articles,in_articles_count
MCO,476,Moody’s,MCO,Commercial Services,Financial Publishing/Services,"Moody's Corporation, often referred to as Mood...",0.675675,"[{'doc_id': 23, 'title': 'Citigroup floats $23...",1


('Communications', 'Specialty Telecommunications')


...

('Utilities', 'Electric Utilities')


,index,stock_name,ticker_symbol,sector,industry,comment,score,in_articles,in_articles_count
NRG,513,NRG Energy,NRG,Utilities,Electric Utilities,"NRG Energy, Inc. is a large American energy co...",0.671679,"[{'doc_id': 1, 'title': 'Ford to build $11.4 b...",5
SO,631,Southern Company,SO,Utilities,Electric Utilities,Southern Company is an American gas and electr...,0.664255,"[{'doc_id': 1, 'title': 'Ford to build $11.4 b...",2
AEP,39,American Electric Power,AEP,Utilities,Electric Utilities,"American Electric Power (AEP), (railcar report...",0.652692,"[{'doc_id': 1, 'title': 'Ford to build $11.4 b...",2
BEP,122,Brookfield Renewable Partners,BEP,Utilities,Electric Utilities,Brookfield Renewable Partners L.P. is a public...,0.653475,"[{'doc_id': 'oos_2', 'title': 'COP26 Climate S...",2
DTE,242,DTE Energy Co,DTE,Utilities,Electric Utilities,DTE Energy (formerly Detroit Edison until 1996...,0.658557,"[{'doc_id': 1, 'title': 'Ford to build $11.4 b...",1
CNP,147,CenterPoint Energy,CNP,Utilities,Electric Utilities,"CenterPoint Energy, Inc. is an American Fortun...",0.665088,"[{'doc_id': 'oos_1', 'title': 'Oil recovers fr...",1
PCG,550,PG&E Corporation,PCG,Utilities,Electric Utilities,The Pacific Gas and Electric Company (PG&E) is...,0.621199,"[{'doc_id': 'oos_2', 'title': 'COP26 Climate S...",1
PEG,568,Public Service Enterprise Group,PEG,Utilities,Electric Utilities,The Public Service Enterprise Group (PSEG) is ...,0.727483,"[{'doc_id': 29, 'title': 'Energy Transfer earn...",1


('Utilities', 'Gas Distributors')


,index,stock_name,ticker_symbol,sector,industry,comment,score,in_articles,in_articles_count
SRE,611,Sempra Energy,SRE,Utilities,Gas Distributors,Sempra is a North American energy infrastructu...,0.625225,"[{'doc_id': 'oos_2', 'title': 'COP26 Climate S...",2


88

In [17]:
for stock in stocks_matches.values():
    print(stock)

{'index': 11, 'stock_name': 'Advanced Micro Devices', 'ticker_symbol': 'AMD', 'sector': 'Electronic Technology', 'industry': 'Semiconductors', 'comment': "Advanced Micro Devices, Inc. (AMD) is an American multinational semiconductor company based in Santa Clara, California, that develops computer processors and related technologies for business and consumer markets. While it initially manufactured its own processors, the company later outsourced its manufacturing, a practice known as going fabless, after GlobalFoundries was spun off in 2009. AMD's main products include microprocessors, motherboard chipsets, embedded processors and graphics processors for servers, workstations, personal computers and embedded system applications."}
{'index': 23, 'stock_name': 'Alibaba', 'ticker_symbol': 'BABA', 'sector': 'Retail Trade', 'industry': 'Internet Retail', 'comment': 'Alibaba Group Holding Limited, also known as Alibaba Group and Alibaba.com, is a Chinese multinational technology company spec

In [19]:
for stock in stocks_matches.values():
    stock["in_articles"] = list()
    stock["not_in_articles"] = list()
    for _, article in articles.items():
        tickers_present = [sm["ticker_symbol"] for sm in article["stock_matches"]]
        article_ = {k:v for k, v in article.items() if k in ["doc_id", "title", "summary"]}
        if stock["ticker_symbol"] in tickers_present:
            stock["in_articles"].append(article_)
        else:
            stock["not_in_articles"].append(article_)


{'index': 11,
 'stock_name': 'Advanced Micro Devices',
 'ticker_symbol': 'AMD',
 'sector': 'Electronic Technology',
 'industry': 'Semiconductors',
 'comment': "Advanced Micro Devices, Inc. (AMD) is an American multinational semiconductor company based in Santa Clara, California, that develops computer processors and related technologies for business and consumer markets. While it initially manufactured its own processors, the company later outsourced its manufacturing, a practice known as going fabless, after GlobalFoundries was spun off in 2009. AMD's main products include microprocessors, motherboard chipsets, embedded processors and graphics processors for servers, workstations, personal computers and embedded system applications.",
 'in_articles': [{'doc_id': 11,
   'title': 'AMD CEO Sees Chip Shortage Easing in 2022',
   'summary': 'AMD CEO Sees Chip Shortage Easing in 2022. The semiconductor shortage will begin to ease in 2022, Advanced Micro Devices CEO Lisa Su said at a session

In [21]:
stocks_matches["AMD"]["in_articles"]

[{'doc_id': 11,
  'title': 'AMD CEO Sees Chip Shortage Easing in 2022',
  'summary': 'AMD CEO Sees Chip Shortage Easing in 2022. The semiconductor shortage will begin to ease in 2022, Advanced Micro Devices CEO Lisa Su said at a session Monday at the Code Conference in Beverly Hills. In an on-stage interview with Code Conference producer Kara Swisher and CNBC tech reporter Jon Fortt, the AMD (ticker: AMD) chief said that the semiconductor industry is investing aggressively in new capacity, with 20 new factories coming online this year and a similar number expected to come online in 2022.'},
 {'doc_id': 12,
  'title': "New Benchmark Leak Reveals Impressive Performance Of Intel's 12900K, 12700K And 12600K Alder Lake Processors",
  'summary': "New Benchmark Leak Reveals Impressive Performance Of Intel's 12900K, 12700K And 12600K Alder Lake Processors. Benchmarks have leaked supposedly revealing the performance of Intel's Core i9-12900K, Core i7-12700K and Core i5-12600K Alder Lake CPUs, c

In [22]:
stocks["AMD"]["in_articles"]

[{'doc_id': 11,
  'title': 'AMD CEO Sees Chip Shortage Easing in 2022',
  'url': 'https://www.barrons.com/articles/amd-ceo-chip-shortage-51632784863',
  'summary': 'AMD CEO Sees Chip Shortage Easing in 2022. The semiconductor shortage will begin to ease in 2022, Advanced Micro Devices CEO Lisa Su said at a session Monday at the Code Conference in Beverly Hills. In an on-stage interview with Code Conference producer Kara Swisher and CNBC tech reporter Jon Fortt, the AMD (ticker: AMD) chief said that the semiconductor industry is investing aggressively in new capacity, with 20 new factories coming online this year and a similar number expected to come online in 2022.'},
 {'doc_id': 12,
  'title': "New Benchmark Leak Reveals Impressive Performance Of Intel's 12900K, 12700K And 12600K Alder Lake Processors",
  'url': 'https://www.forbes.com/sites/antonyleather/2021/09/27/new-benchmark-leak-reveals-impressive-performance-of-intels-12900k-12700k-and-12600k-alder-lake-processors/',
  'summary

### Saving to file

In [31]:
with open("../../data/test/user_study/stocks/stocks.json", "w") as fp:
    json.dump(stocks_matches, fp, ensure_ascii=False, indent="\t")

In [30]:
stocks_matches["T"]

{'index': 72,
 'stock_name': 'AT&T',
 'ticker_symbol': 'T',
 'sector': 'Communications',
 'industry': 'Major Telecommunications',
 'comment': 'AT&T Inc. is an American multinational conglomerate holding company that is Delaware-registered but headquartered at Whitacre Tower in Downtown Dallas, Texas. It is the world\'s largest telecommunications company, it is also the largest provider of mobile telephone services in the U.S. As of 2020, AT&T was ranked 9th on the Fortune 500 rankings of the largest United States corporations, with revenues of $181 billion. The current AT&T reconstitutes much of the former Bell System, and includes four of the seven "Baby Bells" along with the original AT&T Corp., including the long-distance division.',
 'in_articles': [],
 'not_in_articles': [{'doc_id': 0,
   'title': 'AT&T CFO Pascal Desroches Updates Shareholders',
   'summary': 'AT&T CFO Pascal Desroches Updates Shareholders. DALLAS, September 15, 2021--(BUSINESS WIRE)--Pascal Desroches, senior exe